In [1]:
import os
import sys
import datetime

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
from SRModels.loading_methods import load_dataset_as_patches
from SRModels.deep_learning_models.SRCNN_model import SRCNNModel
from SRModels.constants import SRCNN_PATCH_SIZE, SRCNN_STRIDE, RANDOM_SEED

In [2]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
LR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/LR"))
INTERP_MAP_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/interpolation_map.pkl"))

In [3]:
# X -> Low-resolution patches (model input) (Low-resolution images with same size as Y but noised)
# Y -> High-resolution patches (target)
X, Y, hr_h, hr_w = load_dataset_as_patches(HR_ROOT, LR_ROOT, mode="srcnn", patch_size=SRCNN_PATCH_SIZE, stride=SRCNN_STRIDE, interpolation_map_path=INTERP_MAP_PATH)

# Set percentage to keep (e.g., 50% of the data)
keep_pct = 0.5

# Shuffle indices
num_samples = X.shape[0]
indices = np.arange(num_samples)
np.random.seed(RANDOM_SEED)
np.random.shuffle(indices)

# Select the first N indices
N = int(num_samples * keep_pct)
selected_idx = indices[:N]

# Reduce X and Y
X_small = X[selected_idx]
Y_small = Y[selected_idx]

X_train, X_test, Y_train, Y_test = train_test_split(X_small, Y_small, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

In [4]:
print(f"X shape: {X.shape}, Y shape: {Y.shape}")
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X shape: (163296, 48, 48, 3), Y shape: (163296, 48, 48, 3)
X_train shape: (58786, 48, 48, 3), Y_train shape: (58786, 48, 48, 3)
X_val shape: (6532, 48, 48, 3), Y_val shape: (6532, 48, 48, 3)
X_test shape: (16330, 48, 48, 3), Y_test shape: (16330, 48, 48, 3)


In [5]:
model = SRCNNModel()

model.setup_model(input_shape=X_train.shape[1:], learning_rate=1e-4, loss="mean_squared_error")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        15616     
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        2080      
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 3)         2403      
                                                                 
Total params: 20,099
Trainable params: 20,099
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Train SRCNN and capture callbacks for metrics
time_cb, mem_cb = model.fit(
    X_train, Y_train, X_val, Y_val,
    batch_size=64,
    epochs=200,
    use_augmentation=True,
    use_mix=True
)

Training on GPU: /physical_device:GPU:0
Epoch 1/200
919/919 [==============================] - 38s 34ms/step - loss: 0.0228 - psnr: 19.1599 - ssim: 0.3371 - val_loss: 0.0061 - val_psnr: 23.5201 - val_ssim: 0.5424 - lr: 1.0000e-04 - epoch_time_sec: 37.5277 - gpu_mean_current_mb: 172.4012 - gpu_peak_mb: 501.6558
Epoch 2/200
919/919 [==============================] - 40s 44ms/step - loss: 0.0115 - psnr: 20.7165 - ssim: 0.3563 - val_loss: 0.0064 - val_psnr: 23.2022 - val_ssim: 0.5342 - lr: 1.0000e-04 - epoch_time_sec: 40.3421 - gpu_mean_current_mb: 344.7214 - gpu_peak_mb: 503.9307
Epoch 3/200
919/919 [==============================] - 59s 64ms/step - loss: 0.0115 - psnr: 20.7287 - ssim: 0.3571 - val_loss: 0.0062 - val_psnr: 23.5719 - val_ssim: 0.5412 - lr: 1.0000e-04 - epoch_time_sec: 58.8222 - gpu_mean_current_mb: 344.7214 - gpu_peak_mb: 509.0684
Epoch 4/200
919/919 [==============================] - 60s 65ms/step - loss: 0.0113 - psnr: 20.7919 - ssim: 0.3601 - val_loss: 0.0061 - val_psnr

In [7]:
# Evaluate and prepare metrics dictionary
results = model.evaluate(X_test, Y_test)
metrics_dict = {
    "eval_loss": float(results[0]),
    "eval_psnr": float(results[1]),
    "eval_ssim": float(results[2]),
    "epoch_time_sec": time_cb.mean_time_value(),
    "memory": mem_cb.as_dict()
}
print(metrics_dict)

511/511 [==============================] - 3s 6ms/step - loss: 0.0063 - psnr: 23.3914 - ssim: 0.5306
Loss: 0.0063, PSNR: 23.39 dB, SSIM: 0.5306
{'eval_loss': 0.006300096400082111, 'eval_psnr': 23.39139747619629, 'eval_ssim': 0.530609667301178, 'epoch_time_sec': 53.2323264727281, 'memory': {'gpu_mean_current_mb': 329.0560857599432, 'gpu_peak_mb': 512.8701171875}}


In [8]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the trained model
model.save(directory=f"models/SRCNN/SRCNN_{timestamp}", timestamp=timestamp)

# Save hr_h and hr_w for later use in the reconstruction of the images
with open(os.path.abspath(os.path.join(os.getcwd(), f"models/SRCNN/SRCNN_{timestamp}/SRCNN_{timestamp}_hrh_hrw.pkl")), "wb") as f:
    pickle.dump((hr_h, hr_w), f)

Model saved to models/SRCNN/SRCNN_20250908_160141\SRCNN_20250908_160141.h5


In [9]:
# Save evaluation/time/memory metrics next to the model
metrics_path = os.path.abspath(os.path.join(os.getcwd(), f"models/SRCNN/SRCNN_{timestamp}/SRCNN_{timestamp}_metrics.pkl"))

with open(metrics_path, "wb") as f:
    pickle.dump(metrics_dict, f)
    
print(f"Saved metrics to {metrics_path}")

Saved metrics to c:\Users\bgmanuel\InteligenciaArtificial\MasterInteligenciaArtificial\Periodo2\TFM\Super-Resolution-Images-for-3D-Printing-Defect-Detection\SRModels\deep_learning_models\models\SRCNN\SRCNN_20250908_160141\SRCNN_20250908_160141_metrics.pkl
